In [1]:
import os
import numpy as np
import Ngl, Nio
import json
from geopy.distance import distance
import numpy.ma as ma
import sys
import math
import copy
import xarray as xr
from netCDF4 import Dataset
sys.path.append("/mycode/module/")
from module_cal_heating import *
import gc
from module_writenc import *

In [2]:
data = xr.open_dataset("/data/composite3.nc")

In [3]:
u = data["uwind"].sel(lat=slice(-10,30),level=850).isel(time=slice(0,40))
v = data["vwind"].sel(lat=slice(-10,30),level=850).isel(time=slice(0,40))

In [4]:
u_a  =  u.mean("lon").data
v_a  =  v.mean("lon").data

In [5]:
du   =  copy.deepcopy(u.data)
dv   =  copy.deepcopy(u.data)

for tt in range(0,40):
    for latt in range(0,81):
        du[tt,latt,:]  =  u.data[tt,latt,:] - u_a[tt,latt]
        dv[tt,latt,:]  =  v.data[tt,latt,:] - v_a[tt,latt]

In [6]:
lat  =  data["uwind"].lat.sel(lat=slice(-10,30)).data
lon  =  data["uwind"].lon.data

In [7]:
disy = np.array([])
for i in range(0,len(lat)-1):
    disy    =   np.append(disy,distance((lat[i],0),(lat[i+1],0)).m)
location = np.array([0])
for dddd in range(0, len(lat)-1):
    location = np.append(location, np.sum(disy[:dddd + 1]))

In [8]:
u_ay  =  np.gradient(u_a,location,axis=1)

In [9]:
energy = ma.zeros(du.shape)
for tt in range(0,40):
    for latt in range(0,81):
        energy[tt,latt,:] = du[tt,latt,:] * dv[tt,latt,:] * u_ay[tt,latt] * (-1)
        


In [10]:
a_q = {'longname': 'energy_exchange','units': '1','valid_range': [-100000000000000000000000000000000000.0, 1000000000000000000000000000000000.0]}
time = np.arange(0,40)
fout = create_nc_single('/data/','composite-energy_exchange_850',time,lon,lat)
add_variables(fout,'energy',energy,a_q,0)
fout.close()